### Modeling Notebook for Ames, Iowa Housing data. 

### Purpose of notebook is to best predict sale price for homes using multiple features.  

### Question looking to be answered is, "does using data from Ames Iowa, can we construct a model that will be flexible enough to guide home purchases based on specific housing characteristics?"

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [3]:
df = pd.read_csv('./train.csv', index_col="Id")
df.head(1)

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500


In [4]:
"""Setting up features and target variable.  Chosing these specific items from a large list is to
    characterize what I believe based on prior knowledge to be primary factors in housing purchases.
    I understand that plenty more characteristics will play into a better fit, but the goal here
    is to quantify these primary features first with a high bias and create a more balanced model
    over time.
    """
X = df[['Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Overall Qual']]
y = df['SalePrice']

In [5]:
# Discovering the null values within Data set
X.isnull().sum()

Total Bsmt SF    1
1st Flr SF       0
2nd Flr SF       0
Overall Qual     0
dtype: int64

In [6]:
# instead of removing data, I will be filling the few null values with zeros.
X = X.fillna(0)

In [7]:
X.shape

(2051, 4)

In [8]:
# I believe that there is a strong interaction on the Total Square feet of a home not just specific floors.
X['Total_sqft'] = X['Total Bsmt SF'] + X['1st Flr SF'] + X['2nd Flr SF']

X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2051 entries, 109 to 10
Data columns (total 5 columns):
Total Bsmt SF    2051 non-null float64
1st Flr SF       2051 non-null int64
2nd Flr SF       2051 non-null int64
Overall Qual     2051 non-null int64
Total_sqft       2051 non-null float64
dtypes: float64(2), int64(3)
memory usage: 96.1 KB


In [9]:
X.isnull().sum() # null values by series

Total Bsmt SF    0
1st Flr SF       0
2nd Flr SF       0
Overall Qual     0
Total_sqft       0
dtype: int64

In [10]:
# Specifically focusing on the Quality of the home and the Total square feet to determine sales price.
X = X[['Total_sqft', 'Overall Qual']]

In [11]:
# Importance of verifying that a similar shape between target variables and features.
X.shape, y.shape

((2051, 2), (2051,))

In [12]:
# Current mean and standard deviations seems to be within range.  Quality is quantified 0-10 scale.
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Total_sqft,2051.0,2551.289127,809.702102,334.0,2005.5,2456.0,2994.0,11752.0
Overall Qual,2051.0,6.112140,1.426271,1.0,5.0,6.0,7.0,10.0


In [13]:
# Renaming Overall Quality for easier use.
X.rename(columns={'Overall Qual': 'Qual'}, inplace=True)

In [14]:
# Quality will need to be dummied to and overall metrics need to be standardized.
X = pd.get_dummies(X, columns=['Qual'], drop_first=True)

In [15]:
X.head(2)

,Total_sqft,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10
Id,,,,,,,,,,
109,2204.0,0,0,0,0,1,0,0,0,0
544,3035.0,0,0,0,0,0,1,0,0,0


In [16]:
# Importance of train and testing data to ensure success.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1538 entries, 1993 to 2160
Data columns (total 10 columns):
Total_sqft    1538 non-null float64
Qual_2        1538 non-null uint8
Qual_3        1538 non-null uint8
Qual_4        1538 non-null uint8
Qual_5        1538 non-null uint8
Qual_6        1538 non-null uint8
Qual_7        1538 non-null uint8
Qual_8        1538 non-null uint8
Qual_9        1538 non-null uint8
Qual_10       1538 non-null uint8
dtypes: float64(1), uint8(9)
memory usage: 37.5 KB


In [18]:
# Analysis to ensure the data has not adjusted since we set up the training data.
X_train.shape, X_test.shape

((1538, 10), (513, 10))

In [19]:
X_train.head(2)

,Total_sqft,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10
Id,,,,,,,,,,
1993,1848.0,0,0,0,1,0,0,0,0,0
2911,3463.0,0,0,0,0,1,0,0,0,0


In [20]:
# Standardizing the scales so that the dummies can be counted similarly to the total sqft
ss = StandardScaler()
X_tr_scaled = ss.fit_transform(X_train)
X_te_scaled = ss.transform(X_test)

In [21]:
# Using a LinearRegression as our first model.
lr = LinearRegression()

print('CV Score over five folds:', cross_val_score(lr, X_tr_scaled, y_train, cv=5))

print('CV Score mean:', cross_val_score(lr, X_tr_scaled, y_train, cv=5).mean())

CV Score over five folds: [0.82676423 0.713936   0.79354674 0.83860069 0.67610974]
CV Score mean: 0.769791480894316


In [22]:
# setting up the training data tests
lr = LinearRegression()
lr.fit(X_tr_scaled, y_train)
lr.score(X_tr_scaled, y_train)

0.7861765880995701

In [23]:
# R2 score for my test subjects based on the metrics.
lr.score(X_te_scaled, y_test)

0.8061593810337211

## Bringing in the Kaggle Test data

### Setting the Parameters to match those of the Training Data

In [25]:
# Bringing in the data for actual Test.
df_test = pd.read_csv('./test.csv')

df_test.shape

(879, 80)

In [26]:
X1 = df_test[['Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Overall Qual']]

X1 = X1.fillna(0)

In [27]:
X1['Total_sqft'] = X1['Total Bsmt SF'] + X1['1st Flr SF'] + X1['2nd Flr SF']

X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 5 columns):
Total Bsmt SF    879 non-null float64
1st Flr SF       879 non-null int64
2nd Flr SF       879 non-null int64
Overall Qual     879 non-null int64
Total_sqft       879 non-null float64
dtypes: float64(2), int64(3)
memory usage: 34.4 KB


In [28]:
X1 = X1[['Total_sqft', 'Overall Qual']]

X1.head()

,Total_sqft,Overall Qual
0,2948.0,6
1,3934.0,5
2,2150.0,7
3,1936.0,5
4,2788.0,6


In [29]:
X1.rename(columns={'Overall Qual': 'Qual'}, inplace=True)

X1 = pd.get_dummies(X1, columns=['Qual'])

X1.head()

,Total_sqft,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10
0,2948.0,0,0,0,0,1,0,0,0,0
1,3934.0,0,0,0,1,0,0,0,0,0
2,2150.0,0,0,0,0,0,1,0,0,0
3,1936.0,0,0,0,1,0,0,0,0,0
4,2788.0,0,0,0,0,1,0,0,0,0


### Applying the model to the Data

In [30]:
X1_scaled = ss.transform(X1)

In [31]:
predicted = lr.predict(X1_scaled)

predicted.shape, X1_scaled.shape

((879,), (879, 10))

In [32]:
# the data was an array and needed to be converted back to a panda.
X1_scaled = pd.DataFrame(X1_scaled, columns = X_train.columns)

X1_scaled['SalePrice'] = predicted

X1_scaled['Id'] = df_test['Id']

X1_scaled.head()

,Total_sqft,Qual_2,Qual_3,Qual_4,Qual_5,Qual_6,Qual_7,Qual_8,Qual_9,Qual_10,SalePrice,Id
0,0.486583,-0.067618,-0.108821,-0.290906,-0.613924,1.739591,-0.520648,-0.37091,-0.190757,-0.108821,181612.819318,2658
1,1.694478,-0.067618,-0.108821,-0.290906,1.628867,-0.574848,-0.520648,-0.37091,-0.190757,-0.108821,199190.685279,2718
2,-0.491002,-0.067618,-0.108821,-0.290906,-0.613924,-0.574848,1.920683,-0.37091,-0.190757,-0.108821,182571.362478,2414
3,-0.753162,-0.067618,-0.108821,-0.290906,1.628867,-0.574848,-0.520648,-0.37091,-0.190757,-0.108821,125917.917917,1989
4,0.290576,-0.067618,-0.108821,-0.290906,-0.613924,1.739591,-0.520648,-0.37091,-0.190757,-0.108821,175745.130240,625


In [33]:
X1_scaled = X1_scaled.set_index('Id')

X1_scaled = X1_scaled[['SalePrice']]

In [34]:
# X1_scaled.to_csv('submission1_reg_brian_collins.csv')

In [35]:
# Everything looks good!
X1_scaled.head()

,SalePrice
Id,
2658,181612.819318
2718,199190.685279
2414,182571.362478
1989,125917.917917
625,175745.130240
